In [1]:
import tensorflow as tf
import numpy as np
import json
import cv2

In [ ]:
#  Model file for Tensorflow Lite for mobile_face_net
model_file = 'models/mobile_face_net.tflite'

# An input file to load the stored embeddings
input_file = 'embeddings/embeddings.json'

In [ ]:
# Load the embeddings from the JSON file
with open(input_file, 'r') as json_file:
    data = json.load(json_file)
    print(data)

user_name = data['name']
stored_embedding = np.array(data['embedding'])

# Load the TensorFlow Lite model for mobile_face_net
interpreter = tf.lite.Interpreter(model_path=model_file)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
# Initialize the camera feed
cap = cv2.VideoCapture(0)  # 0 for the default camera, change as needed

while True:
    ret, frame = cap.read()
    
    # Detect faces in the camera frame
    # You can use a face detection model here
    
    # Assuming 'faces' is a list of detected faces (each is a bounding box)
    for face_bbox in faces:
        # Extract the face region
        x, y, w, h = face_bbox
        face_image = frame[y:y+h, x:x+w]
        
        # Preprocess the face image for the mobile_face_net model
        face_image = cv2.resize(face_image, (112, 112))
        face_image = (face_image - 127.5) / 128.0  # Normalize pixel values to [-1, 1]
        face_image = np.expand_dims(face_image, axis=0)  # Add batch dimension
        
        # Perform inference with the mobile_face_net model
        interpreter.set_tensor(input_details[0]['index'], face_image)
        interpreter.invoke()
        embeddings = interpreter.get_tensor(output_details[0]['index'])
        
        # Calculate similarity between the detected face and stored embedding
        similarity = np.dot(embeddings, stored_embedding.T)
        
        # Set a threshold for face recognition
        threshold = 0.7  # Adjust as needed
        if similarity > threshold:
            # Look up the individual's name based on individual_id
            
            # Draw a label on the detected face
            cv2.putText(frame, user_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            print(f"User {user_name} face detected!")

    cv2.imshow('Face Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()